In [1]:
import os
import re
import pandas as pd

In [2]:
path = '../data/'

filenames = [os.path.join(path, x) for x in os.listdir(path)]

In [3]:
with open(filenames[0], 'r', encoding='utf-8') as f:
    data = f.read()

In [4]:
# data.split('Página(2)')
lista = re.split(r'Página\(\d+\)', data)

In [5]:
page = pd.DataFrame(lista[2].split('\n'), columns=['text'])

In [6]:
page['main'] = page['text'].str.contains('\d{2}\-\d+\-\d{1}')

In [7]:
page['offset'] = page['text'].shift(-1)

In [8]:
page.loc[page['main']==True, ['text', 'offset']]

,text,offset
14,M 20-31309097-4 DU 31.309.097 CORDOBA ISMAEL ...,...
16,A 20-12697121-5 DU 12.697.121 DIAZ CHAVEZ ROS...,
18,A 30-70990414-7 ZONA IV SRL ...,
20,M 27-04884946-1 LC 04.884.946 ARIAS DOLORES A...,...
22,M 27-05283272-7 LC 05.283.272 PAREDES ROSARIO...,...
24,M 20-06752228-2 LE 06.752.228 RODRIGUEZ FRANC...,...
26,M 20-06770148-9 LE 06.770.148 BETTIO JUAN CAR...,...
28,M 20-06770244-2 DU 06.770.244 CABELLO DAMIAN ...,...
30,M 20-06771704-0 LE 06.771.704 VEGA RAMON LEON...,...
32,A 23-95975398-9 DU 95.975.398 MARTINEZ ESPINO...,...


In [9]:
index, title = page.loc[page['text'].str.contains('TIPO\sNUMERO')].index[0], page.loc[page['text'].str.contains('TIPO\sNUMERO'), 'text'].iloc[0]

In [10]:
slices = pd.DataFrame([i.span() for i in re.finditer('\|', title)], columns=['start', 'finish'])

In [11]:
slices['shift'] = slices['start'].shift(-1)
slices = slices[['finish', 'shift']].dropna().astype(int).to_numpy().tolist()

In [12]:
texts = []
offsets = []

for i, (start, finish) in enumerate(slices):
    text = page.iloc[index]['text']
    offset = page.iloc[index]['offset']
    texts.append(text[start:finish].strip())
    offsets.append(offset[start:finish].strip())

In [23]:
[' '.join(x) for x in zip(texts, offsets)]

['M O',
 ' CUIT',
 'TIPO NUMERO DOCUMENTO',
 'APELLIDO Y NOMBRE O',
 'S E',
 'I V',
 'S E',
 'FECHA NACIM/',
 'D  O  M  I  C  I  L  I  O CALLE - NUMERO - PISO - DPTO  C.POST   LOCALIDAD   CASA PR',
 ' TELEFONO',
 'A C',
 'C A',
 'S I',
 ' USUARIO',
 'NOMBRE USUARIO',
 'FECHA ACTUAL.',
 ' HORA']

In [15]:
for i, (start, finish) in enumerate(slices):
    page[f'text_{i}'] = page['text'].apply(lambda x: x[start:finish])
    page[f'offset_{i}'] = page['offset'].fillna('').apply(lambda x: x[start:finish])

In [18]:
page.loc[page['main']==True, [col for col in page.columns if 'text' in col]]

,text,text_0,text_1,text_2,text_3,text_4,text_5,text_6,text_7,text_8,text_9,text_10,text_11,text_12,text_13,text_14,text_15,text_16
14,M 20-31309097-4 DU 31.309.097 CORDOBA ISMAEL ...,M,20-31309097-4,DU 31.309.097,CORDOBA ISMAEL SEBASTIAN,M,4,1,15/11/1984,PABLO CABRERA 1741 00 5411 SANTA LUC...,,502,2,0,N33276,"CUADRELLI,AN",01/02/2021,11:47:23
16,A 20-12697121-5 DU 12.697.121 DIAZ CHAVEZ ROS...,A,20-12697121-5,DU 12.697.121,DIAZ CHAVEZ ROSIER BERNAR,,0,1,,TUCUMAN SUR 1058 5402 SAN JUAN ...,,000,2,,N36783,GALDEANO ROD,01/02/2021,12:02:46
18,A 30-70990414-7 ZONA IV SRL ...,A,30-70990414-7,,ZONA IV SRL,,0,1,,POSTA DE YATASTO 911 5400 RIVADAVIA...,,000,3,,N36783,GALDEANO ROD,01/02/2021,11:56:50
20,M 27-04884946-1 LC 04.884.946 ARIAS DOLORES A...,M,27-04884946-1,LC 04.884.946,ARIAS DOLORES AGUEDA,F,4,1,18/10/1944,AV.SARMIENTO-E-CONS SIGMAC MB7 5411 SANTA LUC...,542644275126,822,2,1,N36899,"LORCA RIVAS,",01/02/2021,12:15:45
22,M 27-05283272-7 LC 05.283.272 PAREDES ROSARIO...,M,27-05283272-7,LC 05.283.272,PAREDES ROSARIO,F,4,1,01/01/1959,BAIGORRIA B S.ARGENTINO 6984 5407 MARQUESAD...,4331131,821,2,2,N36899,"LORCA RIVAS,",01/02/2021,11:15:56
24,M 20-06752228-2 LE 06.752.228 RODRIGUEZ FRANC...,M,20-06752228-2,LE 06.752.228,RODRIGUEZ FRANCISCO,M,4,1,09/07/1934,MIGUEL RIDAOZ.NORTE 190 5411 SANTA LUC...,,822,2,,N75384,"ESCUDERO,GAB",01/02/2021,09:08:21
26,M 20-06770148-9 LE 06.770.148 BETTIO JUAN CAR...,M,20-06770148-9,LE 06.770.148,BETTIO JUAN CARLOS,M,4,1,22/02/1940,AV LIBERTADOR OESTE 2430 5400 CAPITAL ...,,822,2,1,N75384,"ESCUDERO,GAB",01/02/2021,08:43:22
28,M 20-06770244-2 DU 06.770.244 CABELLO DAMIAN ...,M,20-06770244-2,DU 06.770.244,CABELLO DAMIAN ISMAEL,M,4,1,03/10/1941,ARMANDO GAVIORNO V.IBA#EZ 0 5409 ULLUM ...,,813,2,,N75384,"ESCUDERO,GAB",01/02/2021,09:01:49
30,M 20-06771704-0 LE 06.771.704 VEGA RAMON LEON...,M,20-06771704-0,LE 06.771.704,VEGA RAMON LEONARDO,M,4,1,18/01/1941,RIVADAVIA Y ENTRE RIOS 1 5449 VALLE FER...,,822,2,,N75384,"ESCUDERO,GAB",01/02/2021,08:59:07
32,A 23-95975398-9 DU 95.975.398 MARTINEZ ESPINO...,A,23-95975398-9,DU 95.975.398,MARTINEZ ESPINOZA FELIPE,M,4,1,03/05/1977,GUATEMALA NORTE 0 5400 SAN JUAN ...,0540264-4852336,821,2,,N75384,"ESCUDERO,GAB",01/02/2021,10:45:25


In [19]:
page.loc[page['main']==True, [col for col in page.columns if 'off' in col]]

,offset,offset_0,offset_1,offset_2,offset_3,offset_4,offset_5,offset_6,offset_7,offset_8,offset_9,offset_10,offset_11,offset_12,offset_13,offset_14,offset_15,offset_16
14,...,,,,,,,,,elisma_deejays@outlook.com ...,351555554,,,,,,,
16,,,,,,,,,,,,,,,,,,
18,,,,,,,,,,,,,,,,,,
20,...,,,,,,,,,DOLLYZALAZAR@GMAIL.COM ...,542644715603,,,,,,,
22,...,,,,,,,,,Juliocvivarez@gmail.com,,,,,,,,
24,...,,,,,,,,,NOREPLY@BNA.COM.AR ...,0549264-4852366,,,,,,,
26,...,,,,,,,,,NOREPLY@BNA.COM.AR ...,0549264-5528433,,,,,,,
28,...,,,,,,,,,NOREPLY@BNA.COM.AR ...,0549264-4852336,,,,,,,
30,...,,,,,,,,,NOREPLY@BNA.COM.AR ...,0549264-4852336,,,,,,,
32,...,,,,,,,,,NOREPLY@BNA.COM.AR ...,0549264-4852369,,,,,,,


In [27]:
texts = ['M', 'CUIT', 'DOCUMENTO', 'NOMBRE', 'SEXO', 'I V', 'S E', 'NACIMIENTO', 'DOMICILIO', 'TELEFEONO', 'A C', 'C A', 'S I', 'USUARIO', 'FECHA', 'HORA']

In [29]:
texts = dict(zip([col for col in page.columns if 'text_' in col], texts))

In [30]:
offset = {'offset_8':'EMAIL', 'offset_9':'CELULAR'}

In [37]:
page = page.rename(columns=texts)
page = page.drop(columns=[x for x in [col for col in page.columns if 'offset_' in col] if x not in offset.keys()])
page = page.rename(columns=offset)

In [44]:
page.loc[page['main']==True, [x for x in texts.values()] + [x for x in offset.values()]]

,M,CUIT,DOCUMENTO,NOMBRE,SEXO,I V,S E,NACIMIENTO,DOMICILIO,TELEFEONO,A C,C A,S I,USUARIO,FECHA,HORA,EMAIL,CELULAR
14,M,20-31309097-4,DU 31.309.097,CORDOBA ISMAEL SEBASTIAN,M,4,1,15/11/1984,PABLO CABRERA 1741 00 5411 SANTA LUC...,,502,2,0,N33276,"CUADRELLI,AN",01/02/2021,elisma_deejays@outlook.com ...,351555554
16,A,20-12697121-5,DU 12.697.121,DIAZ CHAVEZ ROSIER BERNAR,,0,1,,TUCUMAN SUR 1058 5402 SAN JUAN ...,,000,2,,N36783,GALDEANO ROD,01/02/2021,,
18,A,30-70990414-7,,ZONA IV SRL,,0,1,,POSTA DE YATASTO 911 5400 RIVADAVIA...,,000,3,,N36783,GALDEANO ROD,01/02/2021,,
20,M,27-04884946-1,LC 04.884.946,ARIAS DOLORES AGUEDA,F,4,1,18/10/1944,AV.SARMIENTO-E-CONS SIGMAC MB7 5411 SANTA LUC...,542644275126,822,2,1,N36899,"LORCA RIVAS,",01/02/2021,DOLLYZALAZAR@GMAIL.COM ...,542644715603
22,M,27-05283272-7,LC 05.283.272,PAREDES ROSARIO,F,4,1,01/01/1959,BAIGORRIA B S.ARGENTINO 6984 5407 MARQUESAD...,4331131,821,2,2,N36899,"LORCA RIVAS,",01/02/2021,Juliocvivarez@gmail.com,
24,M,20-06752228-2,LE 06.752.228,RODRIGUEZ FRANCISCO,M,4,1,09/07/1934,MIGUEL RIDAOZ.NORTE 190 5411 SANTA LUC...,,822,2,,N75384,"ESCUDERO,GAB",01/02/2021,NOREPLY@BNA.COM.AR ...,0549264-4852366
26,M,20-06770148-9,LE 06.770.148,BETTIO JUAN CARLOS,M,4,1,22/02/1940,AV LIBERTADOR OESTE 2430 5400 CAPITAL ...,,822,2,1,N75384,"ESCUDERO,GAB",01/02/2021,NOREPLY@BNA.COM.AR ...,0549264-5528433
28,M,20-06770244-2,DU 06.770.244,CABELLO DAMIAN ISMAEL,M,4,1,03/10/1941,ARMANDO GAVIORNO V.IBA#EZ 0 5409 ULLUM ...,,813,2,,N75384,"ESCUDERO,GAB",01/02/2021,NOREPLY@BNA.COM.AR ...,0549264-4852336
30,M,20-06771704-0,LE 06.771.704,VEGA RAMON LEONARDO,M,4,1,18/01/1941,RIVADAVIA Y ENTRE RIOS 1 5449 VALLE FER...,,822,2,,N75384,"ESCUDERO,GAB",01/02/2021,NOREPLY@BNA.COM.AR ...,0549264-4852336
32,A,23-95975398-9,DU 95.975.398,MARTINEZ ESPINOZA FELIPE,M,4,1,03/05/1977,GUATEMALA NORTE 0 5400 SAN JUAN ...,0540264-4852336,821,2,,N75384,"ESCUDERO,GAB",01/02/2021,NOREPLY@BNA.COM.AR ...,0549264-4852369
